In [2]:
a='vgdewghekqjfndsadv_head.mhd'
a[:-8]

'vgdewghekqjfndsadv_'

In [1]:
import SimpleITK as sitk
import numpy as np
import numpy.ma as ma
import sys
from os import listdir
import glob
from os.path import isfile, join
# glob for the different aquisitions 
#
#input = sys.argv[1]
#patient = sys.argv[2]
#acquisition = sys.argv[3]
#axis = sys.argv[4]


stations = ['head','torso','pelvis','legs','llegs','lllegs', 'feet']
modalities=['ADC','T1','b2000','b1000','b1500','b0']

def is_functional(WB_dict):
    func_dir=[ 1.,0.,-0.,-0.,1.,0.,0., -0.,  1.]
    ana_dir=[ 1. , 0.,  0., -0., -0.,  1.,  0., -1., -0.]
    if np.all(np.around(list(WB_dict.values())[0].get('direction'))==func_dir):
        return True
    elif np.all(np.around(list(WB_dict.values())[0].get('direction'))==ana_dir):
        return False
    else:
        return False
        #raise Exception('The direction of the input file is not correct')
    
def calculate_overlay(WB_dict):
    if is_functional(WB_dict):
        axis=2
    else:
        axis=1
    for i, station in enumerate(list(WB_dict.keys())):
        if i ==0:
            WB_dict.get(station)['overlay']=0
        else:
            originL=WB_dict.get(station).get('origin')
            originH=WB_dict.get(stations[i-1]).get('origin')
            sizeH=WB_dict.get(stations[i-1]).get('size')
            spacingH=WB_dict.get(stations[i-1]).get('spacing')
            overlay = int(np.ceil((sizeH[2]*spacingH[2]-(originH[2]-originL[2]))/spacingH[2]))
            #overlay = np.floor((originL[2]-(originH[2]-spacingH[axis]*sizeH[axis]))/spacingH[axis])
            WB_dict.get(station)['overlay']=overlay
    
    return WB_dict

def get_stations(folder):
    file_names = [f for f in listdir(folder) if isfile(join(folder, f))]
    files=[]
    for station in stations:
        file=glob.glob(folder+'/*'+station+'*'+'.mhd')
        if file !=[]:
            files+=file
    WB_dict=dict()        
    
    for i, file in enumerate(files):
        station=sitk.ReadImage(file)
        origin = station.GetOrigin()
        spacing = station.GetSpacing()
        size = station.GetSize()
        direction = station.GetDirection()
        WB_dict[stations[i]]={'image':station,'origin':origin,'spacing':spacing,'size':size,'direction':direction}
        
        
 #   if is_functional(WB_dict):
  #      WB_dict=dict()
   #     for i, file in enumerate(reversed(files)):
    #        station=sitk.ReadImage(file)
     #       origin = station.GetOrigin()
      #      spacing = station.GetSpacing()
       #     size = station.GetSize()
        #    direction = station.GetDirection()
         #   WB_dict[stations[i]]={'image':station,'origin':origin,'spacing':spacing,'size':size,'direction':direction}
    
    WB_dict=calculate_overlay(WB_dict)
    print(WB_dict)
    return WB_dict

def cumulative_size(WB_dict):
    if is_functional(WB_dict):
        axis=2
    else:
        axis=1
    WB_size=0
    for key , value in WB_dict.items():

        WB_size+=value.get('size')[axis]-value.get('overlay')

    # if direction is functional:

    return WB_size

def get_template_data(WB_dict):
    value=list(WB_dict.values())[0]
    size = value.get('size')
    spacing = value.get('spacing')
    origin = value.get('origin')
    direction = value.get('direction')
    
    return size,spacing,origin,direction

def get_index(i,WB_dict):
    if is_functional(WB_dict):
        axis=2
    else:
        axis=1
    size=0
    overlay=0
    for j in range(i):
        size+=list(WB_dict.values())[j].get('size')[axis]
        if j<i-1:
            overlay+=list(WB_dict.values())[j+1].get('overlay')
        else:
            overlay+=0.5*list(WB_dict.values())[i].get('overlay')
    return size-overlay

def stitch_ana(WB_dict):
    #create an empty template 
    size,spacing,origin,direction=get_template_data(WB_dict)
    cummulativeSize = cumulative_size(WB_dict)
    
    wholeBody = sitk.Image(size[0], int(cummulativeSize), size[2], sitk.sitkFloat32)
    wholeBody.SetSpacing(spacing)
    wholeBody.SetOrigin(origin)
    wholeBody.SetDirection(direction)
    
    #pasting the stations in the empty template
    paste = sitk.PasteImageFilter()
    stations=list(WB_dict.keys())
    for i, station in enumerate(stations):
        image=WB_dict.get(stations[i]).get('image')
        if i ==0:
            paste.SetDestinationIndex([0,0,0])
            paste.SetSourceIndex([0,0,0])
            paste.SetSourceSize(size)
            wholeBody = paste.Execute(wholeBody, image)
        else:
            index=get_index(i,WB_dict)
            sizeL=WB_dict.get(stations[i]).get('size')
            overlay=WB_dict.get(stations[i]).get('overlay')
            
            paste.SetDestinationIndex([0,int(index),0])
            paste.SetSourceIndex([0,int(np.floor(0.5*overlay)),0])
            paste.SetSourceSize([sizeL[0], int(sizeL[1]-np.floor(0.5*overlay)), sizeL[2]])
            
            wholeBody = paste.Execute(wholeBody, image)
     
    return wholeBody

def stitch_func(WB_dict,cummulativeSize):
    #create an empty template 
    size,spacing,origin,direction=get_template_data(WB_dict)
    cummulativeSize = cumulative_size(WB_dict)
    wholeBody = sitk.Image(size[0], size[1], int(cummulativeSize), sitk.sitkFloat32)    
    wholeBody.SetSpacing(spacing)
    wholeBody.SetOrigin(origin)
    wholeBody.SetDirection(direction)
    
    #pasting the stations in the empty template
    paste = sitk.PasteImageFilter()
    stations=list(WB_dict.keys())
    for i, station in enumerate(reversed(stations)):
        image=WB_dict.get(station).get('image')
        if i ==0:
            paste.SetDestinationIndex([0,0,0])
            paste.SetSourceIndex([0,0,0])
            paste.SetSourceSize(size)
            wholeBody = paste.Execute(wholeBody, image)
        else:
            index=get_index(i,WB_dict)
            print(index)
            sizeH=WB_dict.get(station).get('size')
            overlay=WB_dict.get(stations[i-1]).get('overlay')

            paste.SetDestinationIndex([0,0,int(index)])
            paste.SetSourceIndex([0,0, int(np.floor(0.5*overlay))])
            paste.SetSourceSize([sizeH[0], sizeH[1], int(sizeH[2]-np.floor(0.5*overlay))]) 
            wholeBody = paste.Execute(wholeBody, image)
     
    return wholeBody



def linear_interpolation(top,bottom):
    if top.shape != bottom.shape:
        print('The overlay region is not the same size')
        return a
    else:
        c=np.empty(top.shape)
        for i in range(top.shape[1]):
            c[:,i,:]=(-(1/(top.shape[1]-1)*i)+1)*top[:,i,:]+(1/(top.shape[1]-1)*i)*bottom[:,i,:]
        
        return c

def concatenate(images,overlays):
    for i in range(len(images)):
        overlays=list(map(int,overlays))
        #copy the non overlay region
        if i==0:
            print('conc full image')
            image=images[i][:,:-overlays[i],:]
        elif i!=(len(images)-1):
            print('conc half image')
            print(images[i].shape)
            image=np.concatenate((image,images[i][:,overlays[i]:-overlays[i+1],:]),axis=1)
        else:
            print('conc bottom')
            image=np.concatenate((image,images[i][:,overlays[i-1]:,:]),axis=1)
        print('The image size before overlap is')
        print(image.shape)
        if i<len(images)-1:
            overlay=linear_interpolation(images[i][:,-overlays[i]:,:],images[i+1][:,-overlays[i]:,:])
            image=np.concatenate((image,overlay),axis=1)
        print('The image size after overlap is')
        print(image.shape)
    return image
    
    
def interpolate_func(WB_dict,cummulativeSize):
    
    
    return wholeBody

def mask_image_multiply(mask, image):
    components_per_pixel = image.GetNumberOfComponentsPerPixel()
    if  components_per_pixel == 1:
        return mask*image
    else:
        return sitk.Compose([mask*sitk.VectorIndexSelectionCast(image,channel) for channel in range(components_per_pixel)])

def alpha_blend(image1, image2, alpha = 0.5, mask1=None,  mask2=None):
    '''
    Alaph blend two images, pixels can be scalars or vectors.
    The region that is alpha blended is controled by the given masks.
    '''
    
    if not mask1:
        mask1 = sitk.Image(image1.GetSize(), sitk.sitkFloat32) + 1.0
        mask1.CopyInformation(image1)
    else:
        mask1 = sitk.Cast(mask1, sitk.sitkFloat32)
    if not mask2:
        mask2 = sitk.Image(image2.GetSize(),sitk.sitkFloat32) + 1
        mask2.CopyInformation(image2)
    else:        
        mask2 = sitk.Cast(mask2, sitk.sitkFloat32)

    components_per_pixel = image1.GetNumberOfComponentsPerPixel()
    if components_per_pixel>1:
        img1 = sitk.Cast(image1, sitk.sitkVectorFloat32)
        img2 = sitk.Cast(image2, sitk.sitkVectorFloat32)
    else:
        img1 = sitk.Cast(image1, sitk.sitkFloat32)
        img2 = sitk.Cast(image2, sitk.sitkFloat32)
        
    intersection_mask = mask1*mask2
    
    intersection_image = mask_image_multiply(alpha*intersection_mask, img1) + \
                         mask_image_multiply((1-alpha)*intersection_mask, img2)
    return intersection_image + mask_image_multiply(mask2-intersection_mask, img2) + \
           mask_image_multiply(mask1-intersection_mask, img1)

def interpolate_ana(WB_dict):
#create an empty template 
    size,spacing,origin,direction=get_template_data(WB_dict)
    cummulativeSize = cumulative_size(WB_dict)
    
    wholeBody = sitk.Image(size[0], int(cummulativeSize), size[2], sitk.sitkFloat32)
    wholeBody.SetSpacing(spacing)
    wholeBody.SetOrigin(origin)
    wholeBody.SetDirection(direction)
    
    #pasting the stations in the empty template
    paste = sitk.PasteImageFilter()
    stations=list(WB_dict.keys())
    for i, station in enumerate(stations):
        if i < len(stations)-1:
            image=WB_dict.get(stations[i]).get('image')
            index=get_index(i,WB_dict)
            sizeL=WB_dict.get(stations[i]).get('size')
            overlay_prev=WB_dict.get(stations[i]).get('overlay')
            overlay_next=WB_dict.get(stations[i+1]).get('overlay')
            paste.SetDestinationIndex([0,int(index),0])
            paste.SetSourceIndex([0,int(np.floor(overlay_prev)),0])
            paste.SetSourceSize([sizeL[0], int(sizeL[1]-np.floor(overlay_prev)-np.floor(overlay_next)), sizeL[2]])

            #for j in range(overlay_next):
                
            
        else:
            image=WB_dict.get(stations[i]).get('image')
            index=get_index(i,WB_dict)
            sizeL=WB_dict.get(stations[i]).get('size')
            overlay_prev=WB_dict.get(stations[i]).get('overlay')
            overlay_next=0
            paste.SetDestinationIndex([0,int(index),0])
            paste.SetSourceIndex([0,int(np.floor(overlay_prev)),0])
            paste.SetSourceSize([sizeL[0], int(sizeL[1]-np.floor(overlay_prev)-np.floor(overlay_next)), sizeL[2]])
        wholeBody = paste.Execute(wholeBody, image)
    return wholeBody

def constructWholeBody(folder,interpolate):
    ### Load all stations and metadata
    WB_dict=get_stations(folder)

    # calculate final size
    if interpolate:
        if is_functional(WB_dict):
            wholeBody=interpolate_func(WB_dict)
        else:
            wholeBody=interpolate_ana(WB_dict)
     
    else :
        if is_functional(WB_dict):
            wholeBody=stitch_func(WB_dict)
        else:
            wholeBody=stitch_ana(WB_dict)
        

    
    return wholeBody
    
#folder=input+'/Source_T1/'+patient+'/'+acquisition
#wholeBody = constructWholeBody(folder)
#filenameOut = input+'/Source_T1/'+patient+'/'+acquisition+'/Whole_Body_T1.mhd'

# Save whole-body image
#writer = sitk.ImageFileWriter()
#writer.SetFileName(filenameOut)
#writer.Execute(wholeBody)



In [37]:
nifti_list=glob.glob('/Users/joriswuts/Desktop/dataset_0*'+'/*/T1.nii.gz')
for image in nifti_list:
    print(image)
    skeleton=sitk.ReadImage('/Users/joriswuts/Desktop/'+image.split('/')[4]+'/'+image.split('/')[5]+'/Skeleton_Mask_r.nii.gz')
    GT=sitk.ReadImage('/Users/joriswuts/Desktop/'+image.split('/')[4]+'/'+image.split('/')[5]+'/Whole_Body_GT_Man_Clean_r_smoothed.nii.gz')
    im=sitk.ReadImage(image)
    resample = sitk.ResampleImageFilter()
    resample.SetInterpolator = sitk.sitkBSpline

    resample.SetReferenceImage(im)
    
    newimage = resample.Execute(skeleton)
    writer = sitk.ImageFileWriter()
    writer.SetFileName('/Users/joriswuts/Desktop/'+image.split('/')[4]+'/'+image.split('/')[5]+'/Skeleton_Mask_r.nii.gz')
    writer.Execute(newimage)
    
    newimage = resample.Execute(GT)
    writer = sitk.ImageFileWriter()
    writer.SetFileName('/Users/joriswuts/Desktop/'+image.split('/')[4]+'/'+image.split('/')[5]+'/Whole_Body_GT_Man_Clean_r_smoothed.nii.gz')
    writer.Execute(newimage)
    #skeleton.CopyInformation(im)
    #new_im=im*skeleton
    #writer = sitk.ImageFileWriter()
    #writer.SetFileName(image)
    #writer.Execute(new_im)

/Users/joriswuts/Desktop/dataset_04/JSW-027a/T1.nii.gz
/Users/joriswuts/Desktop/dataset_04/CUSL-002/T1.nii.gz
/Users/joriswuts/Desktop/dataset_04/JSW-024a/T1.nii.gz
/Users/joriswuts/Desktop/dataset_04/S027-4/T1.nii.gz
/Users/joriswuts/Desktop/dataset_04/JSW-019a/T1.nii.gz
/Users/joriswuts/Desktop/dataset_04/JSW-020a/T1.nii.gz
/Users/joriswuts/Desktop/dataset_04/S024-3/T1.nii.gz
/Users/joriswuts/Desktop/dataset_04/S024-4/T1.nii.gz
/Users/joriswuts/Desktop/dataset_04/CUSL-029/T1.nii.gz
/Users/joriswuts/Desktop/dataset_04/CUSL-016/T1.nii.gz
/Users/joriswuts/Desktop/dataset_04/JSW-023a/T1.nii.gz
/Users/joriswuts/Desktop/dataset_04/CUSL-018/T1.nii.gz
/Users/joriswuts/Desktop/dataset_04/S024-5/T1.nii.gz
/Users/joriswuts/Desktop/dataset_04/JSW-029a/T1.nii.gz
/Users/joriswuts/Desktop/dataset_04/CUSL-039/T1.nii.gz
/Users/joriswuts/Desktop/dataset_04/JSW-022a/T1.nii.gz
/Users/joriswuts/Desktop/dataset_04/JSW-028a/T1.nii.gz
/Users/joriswuts/Desktop/dataset_04/JSW-021a/T1.nii.gz
/Users/joriswuts/D

SystemError: <built-in function ResampleImageFilter_Execute> returned a result with an error set

In [65]:
del_list=[5]
nifti_list=glob.glob('/Users/joriswuts/Desktop/dataset_04/*/Skeleton*.nii.gz')+glob.glob('/Users/joriswuts/Desktop/dataset_04/*/Whole*.nii.gz')
for image in nifti_list:
    for i in del_list:
        cop='/Users/joriswuts/Desktop/dataset_0'+str(i)+'/'+image.split('/')[5]+'/'+image.split('/')[6]
        shutil.copyfile(image,cop)
  

In [86]:
for fail in fails:
    skel_reshape_list=glob.glob('/Users/joriswuts/Desktop/dataset_05/'+str(fail)+'/*rb1000.nii.gz')
    im=sitk.ReadImage('/Users/joriswuts/Desktop/dataset_05/'+str(fail)+'/T1.nii.gz')
    resample = sitk.ResampleImageFilter()
    resample.SetInterpolator = sitk.sitkBSpline

    resample.SetReferenceImage(im)
    for im_r in skel_reshape_list:
        res=sitk.ReadImage(im_r)
        newimage = resample.Execute(res)
        if res.GetSize()!=newimage.GetSize():
            print('eureka')
            writer = sitk.ImageFileWriter()
            writer.SetFileName(im_r)
            writer.Execute(newimage)

eureka
eureka
eureka
eureka
eureka
eureka


In [87]:
#image=sitk.ReadImage('/Users/joriswuts/Desktop/dataset_03/JSW-020a/DixonIP.mhd')
#fails=['JSW-028a','JSW-021a','CUSL-007','JSW-030a','JSW-025a']
import os
import shutil
nifti_list=glob.glob('/Users/joriswuts/Desktop/dataset_05'+'/*/*.nii.gz')
prev='JSW-00'
check=True
for image in nifti_list:
    print(image)

    if check:
        print(image)
        
        if image.split('/')[5] in fails:
            im=sitk.ReadImage(image)
            im=sitk.Cast(im,sitk.sitkFloat32)
            print(image.split('/')[5])
            try:
                if image.split('/')[5]!=prev:

                    prev=image.split('/')[5]
                    print(prev)
                    skeleton=sitk.ReadImage('/Users/joriswuts/Desktop/dataset_05/'+image.split('/')[5]+'/Skeleton_Mask_r.nii.gz')

                    im=sitk.ReadImage(image)
                    im=sitk.Cast(im,sitk.sitkFloat32)

                    skeleton=sitk.Cast(skeleton,sitk.sitkUInt8)

                    closing = sitk.BinaryDilateImageFilter()
                    closing.SetKernelRadius(30)
                    skeleton = closing.Execute(skeleton)


                    closing = sitk.BinaryErodeImageFilter()
                    closing.SetKernelRadius(10)
                    skeleton = closing.Execute(skeleton)
                    skeleton=sitk.Cast(skeleton,sitk.sitkFloat32)

                skeleton.CopyInformation(im)

                new_im=im*skeleton
                writer = sitk.ImageFileWriter()
                writer.SetFileName(image)
                writer.Execute(new_im)
                #os.remove(image)
            except:
                print('this image has failed'+str(image.split('/')[5]))
                fails.append(image.split('/')[5])

/Users/joriswuts/Desktop/dataset_05/JSW-027a/Whole_Body_GT_Man_Clean_r_smoothed.nii.gz
/Users/joriswuts/Desktop/dataset_05/JSW-027a/Whole_Body_GT_Man_Clean_r_smoothed.nii.gz
/Users/joriswuts/Desktop/dataset_05/JSW-027a/Skeleton_Mask_r.nii.gz
/Users/joriswuts/Desktop/dataset_05/JSW-027a/Skeleton_Mask_r.nii.gz
/Users/joriswuts/Desktop/dataset_05/JSW-027a/rADC.nii.gz
/Users/joriswuts/Desktop/dataset_05/JSW-027a/rADC.nii.gz
/Users/joriswuts/Desktop/dataset_05/JSW-027a/rb1000.nii.gz
/Users/joriswuts/Desktop/dataset_05/JSW-027a/rb1000.nii.gz
/Users/joriswuts/Desktop/dataset_05/JSW-027a/T1.nii.gz
/Users/joriswuts/Desktop/dataset_05/JSW-027a/T1.nii.gz
/Users/joriswuts/Desktop/dataset_05/CUSL-002/Whole_Body_GT_Man_Clean_r_smoothed.nii.gz
/Users/joriswuts/Desktop/dataset_05/CUSL-002/Whole_Body_GT_Man_Clean_r_smoothed.nii.gz
/Users/joriswuts/Desktop/dataset_05/CUSL-002/Skeleton_Mask_r.nii.gz
/Users/joriswuts/Desktop/dataset_05/CUSL-002/Skeleton_Mask_r.nii.gz
/Users/joriswuts/Desktop/dataset_05/CU

JSW-028a
JSW-028a
/Users/joriswuts/Desktop/dataset_05/JSW-028a/Skeleton_Mask_r.nii.gz
/Users/joriswuts/Desktop/dataset_05/JSW-028a/Skeleton_Mask_r.nii.gz
JSW-028a
/Users/joriswuts/Desktop/dataset_05/JSW-028a/rADC.nii.gz
/Users/joriswuts/Desktop/dataset_05/JSW-028a/rADC.nii.gz
JSW-028a
/Users/joriswuts/Desktop/dataset_05/JSW-028a/rb1000.nii.gz
/Users/joriswuts/Desktop/dataset_05/JSW-028a/rb1000.nii.gz
JSW-028a
/Users/joriswuts/Desktop/dataset_05/JSW-028a/T1.nii.gz
/Users/joriswuts/Desktop/dataset_05/JSW-028a/T1.nii.gz
JSW-028a
/Users/joriswuts/Desktop/dataset_05/JSW-021a/Whole_Body_GT_Man_Clean_r_smoothed.nii.gz
/Users/joriswuts/Desktop/dataset_05/JSW-021a/Whole_Body_GT_Man_Clean_r_smoothed.nii.gz
JSW-021a
JSW-021a
/Users/joriswuts/Desktop/dataset_05/JSW-021a/Skeleton_Mask_r.nii.gz
/Users/joriswuts/Desktop/dataset_05/JSW-021a/Skeleton_Mask_r.nii.gz
JSW-021a
/Users/joriswuts/Desktop/dataset_05/JSW-021a/rADC.nii.gz
/Users/joriswuts/Desktop/dataset_05/JSW-021a/rADC.nii.gz
JSW-021a
/Users/j

CUSL-024
CUSL-024
/Users/joriswuts/Desktop/dataset_05/CUSL-024/Skeleton_Mask_r.nii.gz
/Users/joriswuts/Desktop/dataset_05/CUSL-024/Skeleton_Mask_r.nii.gz
CUSL-024
/Users/joriswuts/Desktop/dataset_05/CUSL-024/rADC.nii.gz
/Users/joriswuts/Desktop/dataset_05/CUSL-024/rADC.nii.gz
CUSL-024
/Users/joriswuts/Desktop/dataset_05/CUSL-024/rb1000.nii.gz
/Users/joriswuts/Desktop/dataset_05/CUSL-024/rb1000.nii.gz
CUSL-024
/Users/joriswuts/Desktop/dataset_05/CUSL-024/T1.nii.gz
/Users/joriswuts/Desktop/dataset_05/CUSL-024/T1.nii.gz
CUSL-024


In [85]:
fails

['JSW-028a',
 'JSW-021a',
 'CUSL-007',
 'JSW-030a',
 'JSW-025a',
 'CUSL-022',
 'CUSL-024']

In [ ]:
import os
def register_image2image(movingImage,fixedImage):
    
    fixedImage=sitk.ReadImage(fixedImage)
    movingImage=sitk.ReadImage(movingImage)
    elastixImageFilter=sitk.ElastixImageFilter()
    elastixImageFilter.LogToConsoleOff()
    writer = sitk.ImageFileWriter()
    #fixedImageMask=fixedImage>-100000
    fixedImage.SetDirection(movingImage.GetDirection())
    #fixedImageMask.SetSpacing(fixedImage.GetSpacing())
    parameterMapVector = sitk.VectorOfParameterMap()
    elastixImageFilter.SetFixedImage(fixedImage)
    elastixImageFilter.SetMovingImage(movingImage)
    #elastixImageFilter.SetFixedMask(fixedImageMask)
    parameterMapVector.append(sitk.ReadParameterFile(
                        '/Users/joriswuts/Clean_Code_versino_IMAC/01_PREPROCESSING/Euler_S2S_MSD2.txt'))
    parameterMapVector.append(sitk.ReadParameterFile(
                        '/Users/joriswuts/Clean_Code_versino_IMAC/01_PREPROCESSING/S2A_Pair_BSpline_WB2.txt'))    
    elastixImageFilter.SetParameterMap(parameterMapVector)
    elastixImageFilter.Execute()

    parammap=elastixImageFilter.GetTransformParameterMap()[0]
    return parammap 
patients=['/Users/joriswuts/Desktop/dataset_01/S033-2 copy','/Users/joriswuts/Desktop/dataset_01/S034-2 copy','/Users/joriswuts/Desktop/dataset_01/S045-2 copy']
for patient in patients:
    fixed_image=patient+'/T1.nii.gz'
    moving_image=patient+'/Whole_Body_T1.mhd'
    if os.path.isfile(moving_image):
        parammap=register_image2image(moving_image,fixed_image)
        writer = sitk.ImageFileWriter()
        transformixImageFilter=sitk.TransformixImageFilter()
        transformixImageFilter.SetTransformParameterMap(parammap)
        #transformixImageFilter.LogToConsoleOff()
        # all images including the original one 
        func_modalities=glob.glob(patient+'/*Skeleton*.nii.gz')+glob.glob(patient+'/*smoothed*.nii.gz')
        for modality in func_modalities:

            image_path=modality
            movingImage=sitk.ReadImage(image_path)
            transformixImageFilter.SetMovingImage(movingImage)
            transformixImageFilter.Execute()
            resultImage=transformixImageFilter.GetResultImage()
            resultImage=resultImage>0.5
            #resultImage.clip(0,max)
            writer.SetFileName(image_path)
            writer.Execute(resultImage)

In [31]:
a='/Users/joriswuts/Desktop/dataset_03/JSW-027a/rb1000.mhd'
a[:-3]+'nii'

'/Users/joriswuts/Desktop/dataset_03/JSW-027a/rb1000.nii'

In [21]:
import numpy as np

import SimpleITK as sitk


resample = sitk.ResampleImageFilter()
resample.SetInterpolator(sitk.sitkLinear)
resample.SetOutputDirection(image.GetDirection())
resample.SetOutputOrigin(image.GetOrigin())
new_spacing = [1, 1, 1]
resample.SetOutputSpacing(new_spacing)

orig_size = np.array(image.GetSize(), dtype=np.int)
orig_spacing = image.GetSpacing()
new_size = [osize*(ospac/newspac) for osize,ospac,newspac in zip(orig_size,orig_spacing,new_spacing)]
new_size = [np.ceil(val).astype(np.int) for val in new_size] #  Image dimensions are in integers
new_size = [int(s) for s in new_size]
resample.SetSize(new_size)

newimage = resample.Execute(image)

In [22]:
writer = sitk.ImageFileWriter()
writer.SetFileName('/Users/joriswuts/Desktop/dataset_03/JSW-020a/DixonIP_resampled.mhd')
writer.Execute(newimage)

In [11]:
func_dir=[ 1.,0.,-0.,-0.,1.,0.,0., -0.,  1.]

In [23]:
[[1,2,3],[4,5,6]][0]

[1, 2, 3]

In [26]:
folder='/Users/joriswuts/Desktop/dataset_03'
modality_list =[glob.glob(folder+'/*/*b1000.mhd'),glob.glob(folder+'/*/*T1.mhd')+glob.glob(folder+'/*/*DixonIP.mhd')]
for file in modality_list[0]:
    print(file)

/Users/joriswuts/Desktop/dataset_03/JSW-027a/rb1000.mhd
/Users/joriswuts/Desktop/dataset_03/CUSL-002/rb1000.mhd
/Users/joriswuts/Desktop/dataset_03/JSW-024a/rb1000.mhd
/Users/joriswuts/Desktop/dataset_03/S033-2/rb1000.mhd
/Users/joriswuts/Desktop/dataset_03/S027-4/rb1000.mhd
/Users/joriswuts/Desktop/dataset_03/JSW-019a/rb1000.mhd
/Users/joriswuts/Desktop/dataset_03/S034-2/rb1000.mhd
/Users/joriswuts/Desktop/dataset_03/JSW-020a/rb1000.mhd
/Users/joriswuts/Desktop/dataset_03/S024-3/rb1000.mhd
/Users/joriswuts/Desktop/dataset_03/S024-4/b1000.mhd
/Users/joriswuts/Desktop/dataset_03/CUSL-029/rb1000.mhd
/Users/joriswuts/Desktop/dataset_03/CUSL-016/rb1000.mhd
/Users/joriswuts/Desktop/dataset_03/JSW-023a/rb1000.mhd
/Users/joriswuts/Desktop/dataset_03/CUSL-018/rb1000.mhd
/Users/joriswuts/Desktop/dataset_03/S024-5/rb1000.mhd
/Users/joriswuts/Desktop/dataset_03/JSW-029a/rb1000.mhd
/Users/joriswuts/Desktop/dataset_03/S045-2/rb1000.mhd
/Users/joriswuts/Desktop/dataset_03/CUSL-039/rb1000.mhd
/Users/

In [2]:
elastixImageFilter=sitk.ElastixImageFilter()

AttributeError: module 'SimpleITK' has no attribute 'ElastixImageFilter'

In [1]:
for i in range(1):
    print(i)

0


In [2]:
a=[1,2,3]

In [3]:
a

[1, 2, 3]

In [4]:
a+[]

[1, 2, 3]

In [5]:
a='gfyuewbv'
'g' in a 

True

In [18]:
a='DWIBS b0-50'


In [19]:
a[0:4]

'DWIB'